In [ ]:
import face_recognition
import cv2
import numpy as np
import pandas as pd

In [ ]:
face_data = pd.read_csv("facedata.csv")

In [ ]:
face_data

# Load a sample picture and learn how to recognize it.

In [ ]:
known_face_encodings = []
for i in range(len(face_data["Name"])):
    image = face_recognition.load_image_file("%d.jpg"%(i))
    face_encoding = face_recognition.face_encodings(image)[0]
    
    # Create arrays of known face encodings and their names
    known_face_encodings.append(face_encoding)


known_face_names = face_data["Name"].values

# Face Unlock

In [ ]:
#csv file for recording data whenever this face unlock is used

face_data_record = pd.read_csv("facedatarecord.csv")

In [ ]:
import datetime

video_capture = cv2.VideoCapture(0)

name = "Unknown"

# Initialize some variables
face_locations = []
face_encodings = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            
            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_data_record = face_data_record.append({'Name': name, 'Time': datetime.datetime.now()}, ignore_index=True)
            

    process_this_frame = not process_this_frame


    cv2.imshow('Video', frame)
    
    if name != "Unknown":
        
        face_data.to_csv("facedata.csv",index=False)
        # your unlock task
        break
        

    
        
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()